In [ ]:
# attempt to test best blended model with 25 runs similarly to how the author originally did
# use a 90/10 train/test split and evaluate performance of 25 runs, each with different random samples


import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load datasets
main_data = pd.read_csv("./data/train.csv")  # Superconductivity dataset
unique_m = pd.read_csv("./data/unique_m.csv")

# Remove 'critical_temp' from unique_m to avoid duplication
unique_m = unique_m.drop(columns=["critical_temp"], errors='ignore')

# Merge datasets assuming rows align (index-based merge)
merged_data = pd.concat([main_data, unique_m], axis=1)

# Feature Engineering: Physics-Based Ratio, Thermal Conductivity Transformation, Log transformation
merged_data["mass_density_ratio"] = merged_data["wtd_mean_atomic_mass"] / (merged_data["wtd_mean_Density"] + 1e-9)
merged_data["affinity_valence_ratio"] = merged_data["wtd_mean_ElectronAffinity"] / (merged_data["wtd_mean_Valence"] + 1e-9)
merged_data["log_thermal_conductivity"] = np.log1p(merged_data["range_ThermalConductivity"])

# Define target and features
target = "critical_temp"
features = ['mean_atomic_mass', 'wtd_mean_atomic_mass', 'gmean_atomic_mass',
       'entropy_atomic_mass', 'wtd_entropy_atomic_mass', 'range_atomic_mass',
       'wtd_range_atomic_mass', 'wtd_std_atomic_mass', 'mean_fie',
       'wtd_mean_fie', 'wtd_entropy_fie', 'range_fie', 'wtd_range_fie',
       'wtd_std_fie', 'mean_atomic_radius', 'wtd_mean_atomic_radius',
       'gmean_atomic_radius', 'range_atomic_radius', 'wtd_range_atomic_radius',
       'mean_Density', 'wtd_mean_Density', 'gmean_Density', 'entropy_Density',
       'wtd_entropy_Density', 'range_Density', 'wtd_range_Density',
       'wtd_std_Density', 'mean_ElectronAffinity', 'wtd_mean_ElectronAffinity',
       'gmean_ElectronAffinity', 'wtd_gmean_ElectronAffinity',
       'entropy_ElectronAffinity', 'wtd_entropy_ElectronAffinity',
       'range_ElectronAffinity', 'wtd_range_ElectronAffinity',
       'wtd_std_ElectronAffinity', 'mean_FusionHeat', 'wtd_mean_FusionHeat',
       'gmean_FusionHeat', 'entropy_FusionHeat', 'wtd_entropy_FusionHeat',
       'range_FusionHeat', 'wtd_range_FusionHeat', 'wtd_std_FusionHeat',
       'mean_ThermalConductivity', 'wtd_mean_ThermalConductivity',
       'gmean_ThermalConductivity', 'wtd_gmean_ThermalConductivity',
       'entropy_ThermalConductivity', 'wtd_entropy_ThermalConductivity',
       'range_ThermalConductivity', 'wtd_range_ThermalConductivity',
       'mean_Valence', 'wtd_mean_Valence', 'range_Valence',
       'wtd_range_Valence', 'wtd_std_Valence', 'H', 'B', 'C', 'O', 'F', 'Na',
       'Mg', 'Al', 'Cl', 'K', 'Ca', 'V', 'Cr', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
       'As', 'Se', 'Sr', 'Y', 'Nb', 'Sn', 'I', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
       'Sm', 'Eu', 'Gd', 'Tb', 'Yb', 'Hg', 'Tl', 'Pb', 'Bi',
       'mass_density_ratio', 'affinity_valence_ratio',
       'log_thermal_conductivity']
X = merged_data[features]
y = merged_data[target]


# Optimized LightGBM Model
optimized_lgb = lgb.LGBMRegressor(n_estimators=496, max_depth=15, learning_rate=0.057878589503943714, 
                                  subsample=0.6619352139576826, colsample_bytree=0.7512301369524537, 
                                  num_leaves=148, force_col_wise=True, verbose=-1, random_state=42)


# Optimized XGBoost Model
optimized_xgb = xgb.XGBRegressor(n_estimators=407, max_depth=10, learning_rate=0.02962746174406205,
                                 subsample=0.8786056663685927, colsample_bytree=0.6260167856358314,
                                 gamma=4.321388407974591, tree_method="hist", random_state=42)


# Define blending weights
best_weight_lgb = 0.3454  # Previously found optimal weight
best_weight_xgb = 1.0 - best_weight_lgb


# prepare for doing 25 runs as in the original paper
n_runs = 25
rmse_list = []
r2_list = []

for i in range(n_runs):
    # Perform a 90/10 random split; vary the random_state for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42 + i)
    
    # Fit the model on the training set
    optimized_lgb.fit(X_train, y_train)
    optimized_xgb.fit(X_train, y_train)

    # Predict on the test set
    y_pred_lgb_test = optimized_lgb.predict(X_test)
    y_pred_xgb_test = optimized_xgb.predict(X_test)
    y_pred = (best_weight_lgb * y_pred_lgb_test) + (best_weight_xgb * y_pred_xgb_test)
    
    # Compute RMSE and R² for this run
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    rmse_list.append(rmse)
    r2_list.append(r2)
    
    print(f"Run {i+1}: RMSE = {rmse:.4f}, R² = {r2:.4f}")

# Compute the average RMSE and R² over the 25 runs
avg_rmse = np.mean(rmse_list)
avg_r2 = np.mean(r2_list)
print(f"\nAverage RMSE over 25 runs: {avg_rmse:.4f}")
print(f"Average R² over 25 runs: {avg_r2:.4f}")




Run 1: RMSE = 8.2668, R² = 0.9404
Run 2: RMSE = 8.8676, R² = 0.9313
Run 3: RMSE = 8.6958, R² = 0.9343
Run 4: RMSE = 8.4911, R² = 0.9378
Run 5: RMSE = 9.1397, R² = 0.9275
Run 6: RMSE = 8.6022, R² = 0.9343
Run 7: RMSE = 8.9883, R² = 0.9315
Run 8: RMSE = 8.7249, R² = 0.9343
Run 9: RMSE = 8.2244, R² = 0.9427
Run 10: RMSE = 8.3890, R² = 0.9406
Run 11: RMSE = 8.9937, R² = 0.9331
Run 12: RMSE = 9.7432, R² = 0.9218
Run 13: RMSE = 8.6977, R² = 0.9356
Run 14: RMSE = 8.2152, R² = 0.9420
Run 15: RMSE = 8.6335, R² = 0.9383
Run 16: RMSE = 9.1838, R² = 0.9267
Run 17: RMSE = 8.5961, R² = 0.9371
Run 18: RMSE = 8.5562, R² = 0.9376
Run 19: RMSE = 9.2510, R² = 0.9254
Run 20: RMSE = 8.6889, R² = 0.9342
Run 21: RMSE = 8.9863, R² = 0.9315
Run 22: RMSE = 9.6585, R² = 0.9206
Run 23: RMSE = 8.6251, R² = 0.9372
Run 24: RMSE = 9.3573, R² = 0.9265
Run 25: RMSE = 8.4191, R² = 0.9395

Average RMSE over 25 runs: 8.7998
Average R² over 25 runs: 0.9337


Results with 90/10 split:

Run 1: RMSE = 8.2668, R² = 0.9404
Run 2: RMSE = 8.8676, R² = 0.9313
Run 3: RMSE = 8.6958, R² = 0.9343
Run 4: RMSE = 8.4911, R² = 0.9378
Run 5: RMSE = 9.1397, R² = 0.9275
Run 6: RMSE = 8.6022, R² = 0.9343
Run 7: RMSE = 8.9883, R² = 0.9315
Run 8: RMSE = 8.7249, R² = 0.9343
Run 9: RMSE = 8.2244, R² = 0.9427
Run 10: RMSE = 8.3890, R² = 0.9406
Run 11: RMSE = 8.9937, R² = 0.9331
Run 12: RMSE = 9.7432, R² = 0.9218
Run 13: RMSE = 8.6977, R² = 0.9356
Run 14: RMSE = 8.2152, R² = 0.9420
Run 15: RMSE = 8.6335, R² = 0.9383
Run 16: RMSE = 9.1838, R² = 0.9267
Run 17: RMSE = 8.5961, R² = 0.9371
Run 18: RMSE = 8.5562, R² = 0.9376
Run 19: RMSE = 9.2510, R² = 0.9254
Run 20: RMSE = 8.6889, R² = 0.9342
Run 21: RMSE = 8.9863, R² = 0.9315
Run 22: RMSE = 9.6585, R² = 0.9206
Run 23: RMSE = 8.6251, R² = 0.9372
Run 24: RMSE = 9.3573, R² = 0.9265
Run 25: RMSE = 8.4191, R² = 0.9395

Average RMSE over 25 runs: 8.7998

Average R² over 25 runs: 0.9337

In [ ]:
# now do it with 66/33 split

# prepare for doing 25 runs as in the original paper
n_runs = 25
rmse_list = []
r2_list = []

for i in range(n_runs):
    # Perform a 66/33 random split; vary the random_state for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42 + i)
    
    # Fit the model on the training set
    optimized_lgb.fit(X_train, y_train)
    optimized_xgb.fit(X_train, y_train)

    # Predict on the test set
    y_pred_lgb_test = optimized_lgb.predict(X_test)
    y_pred_xgb_test = optimized_xgb.predict(X_test)
    y_pred = (best_weight_lgb * y_pred_lgb_test) + (best_weight_xgb * y_pred_xgb_test)
    
    # Compute RMSE and R² for this run
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    rmse_list.append(rmse)
    r2_list.append(r2)
    
    print(f"Run {i+1}: RMSE = {rmse:.4f}, R² = {r2:.4f}")

# Compute the average RMSE and R² over the 25 runs
avg_rmse = np.mean(rmse_list)
avg_r2 = np.mean(r2_list)
print(f"\nAverage RMSE over 25 runs: {avg_rmse:.4f}")
print(f"Average R² over 25 runs: {avg_r2:.4f}")

Run 1: RMSE = 9.1013, R² = 0.9288
Run 2: RMSE = 9.1598, R² = 0.9283
Run 3: RMSE = 8.9797, R² = 0.9312
Run 4: RMSE = 9.1756, R² = 0.9278
Run 5: RMSE = 9.3386, R² = 0.9252
Run 6: RMSE = 9.2190, R² = 0.9282
Run 7: RMSE = 9.3420, R² = 0.9253
Run 8: RMSE = 9.1238, R² = 0.9280
Run 9: RMSE = 9.0236, R² = 0.9306
Run 10: RMSE = 9.3054, R² = 0.9268
Run 11: RMSE = 9.0739, R² = 0.9298
Run 12: RMSE = 9.2928, R² = 0.9281
Run 13: RMSE = 9.1072, R² = 0.9284
Run 14: RMSE = 8.8354, R² = 0.9336
Run 15: RMSE = 8.9876, R² = 0.9313
Run 16: RMSE = 9.0327, R² = 0.9307
Run 17: RMSE = 9.0940, R² = 0.9295
Run 18: RMSE = 9.2818, R² = 0.9269
Run 19: RMSE = 9.0650, R² = 0.9303
Run 20: RMSE = 9.0698, R² = 0.9301
Run 21: RMSE = 9.3236, R² = 0.9267
Run 22: RMSE = 9.3789, R² = 0.9249
Run 23: RMSE = 9.1169, R² = 0.9284
Run 24: RMSE = 9.3344, R² = 0.9258
Run 25: RMSE = 9.2138, R² = 0.9274

Average RMSE over 25 runs: 9.1591
Average R² over 25 runs: 0.9285


Results 33/66 split 25 runs:

Run 1: RMSE = 9.1013, R² = 0.9288
Run 2: RMSE = 9.1598, R² = 0.9283
Run 3: RMSE = 8.9797, R² = 0.9312
Run 4: RMSE = 9.1756, R² = 0.9278
Run 5: RMSE = 9.3386, R² = 0.9252
Run 6: RMSE = 9.2190, R² = 0.9282
Run 7: RMSE = 9.3420, R² = 0.9253
Run 8: RMSE = 9.1238, R² = 0.9280
Run 9: RMSE = 9.0236, R² = 0.9306
Run 10: RMSE = 9.3054, R² = 0.9268
Run 11: RMSE = 9.0739, R² = 0.9298
Run 12: RMSE = 9.2928, R² = 0.9281
Run 13: RMSE = 9.1072, R² = 0.9284
Run 14: RMSE = 8.8354, R² = 0.9336
Run 15: RMSE = 8.9876, R² = 0.9313
Run 16: RMSE = 9.0327, R² = 0.9307
Run 17: RMSE = 9.0940, R² = 0.9295
Run 18: RMSE = 9.2818, R² = 0.9269
Run 19: RMSE = 9.0650, R² = 0.9303
Run 20: RMSE = 9.0698, R² = 0.9301
Run 21: RMSE = 9.3236, R² = 0.9267
Run 22: RMSE = 9.3789, R² = 0.9249
Run 23: RMSE = 9.1169, R² = 0.9284
Run 24: RMSE = 9.3344, R² = 0.9258
Run 25: RMSE = 9.2138, R² = 0.9274

Average RMSE over 25 runs: 9.1591

Average R² over 25 runs: 0.9285

In [ ]:
# now train and test the new model ONLY on the original features
# use a 90/10 train/test split and evaluate performance of 25 runs, each with different random samples


import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load the data
main_data = pd.read_csv("./data/train.csv")

# 'critical_temp' is the target
X = main_data.drop('critical_temp', axis=1)
y = main_data['critical_temp']


# Optimized LightGBM Model
optimized_lgb = lgb.LGBMRegressor(n_estimators=496, max_depth=15, learning_rate=0.057878589503943714, 
                                  subsample=0.6619352139576826, colsample_bytree=0.7512301369524537, 
                                  num_leaves=148, force_col_wise=True, verbose=-1, random_state=42)


# Optimized XGBoost Model
optimized_xgb = xgb.XGBRegressor(n_estimators=407, max_depth=10, learning_rate=0.02962746174406205,
                                 subsample=0.8786056663685927, colsample_bytree=0.6260167856358314,
                                 gamma=4.321388407974591, tree_method="hist", random_state=42)


# Define blending weights
best_weight_lgb = 0.3454  # Previously found optimal weight
best_weight_xgb = 1.0 - best_weight_lgb


# prepare for doing 25 runs as in the original paper
n_runs = 25
rmse_list = []
r2_list = []

for i in range(n_runs):
    # Perform a 90/10 random split; vary the random_state for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42 + i)
    
    # Fit the model on the training set
    optimized_lgb.fit(X_train, y_train)
    optimized_xgb.fit(X_train, y_train)

    # Predict on the test set
    y_pred_lgb_test = optimized_lgb.predict(X_test)
    y_pred_xgb_test = optimized_xgb.predict(X_test)
    y_pred = (best_weight_lgb * y_pred_lgb_test) + (best_weight_xgb * y_pred_xgb_test)
    
    # Compute RMSE and R² for this run
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    rmse_list.append(rmse)
    r2_list.append(r2)
    
    print(f"Run {i+1}: RMSE = {rmse:.4f}, R² = {r2:.4f}")

# Compute the average RMSE and R² over the 25 runs
avg_rmse = np.mean(rmse_list)
avg_r2 = np.mean(r2_list)
print(f"\nAverage RMSE over 25 runs: {avg_rmse:.4f}")
print(f"Average R² over 25 runs: {avg_r2:.4f}")




Run 1: RMSE = 8.3812, R² = 0.9387
Run 2: RMSE = 9.0486, R² = 0.9285
Run 3: RMSE = 8.8819, R² = 0.9314
Run 4: RMSE = 8.6493, R² = 0.9355
Run 5: RMSE = 9.3036, R² = 0.9249
Run 6: RMSE = 8.8660, R² = 0.9303
Run 7: RMSE = 8.9372, R² = 0.9323
Run 8: RMSE = 8.8508, R² = 0.9324
Run 9: RMSE = 8.3508, R² = 0.9409
Run 10: RMSE = 8.3315, R² = 0.9414
Run 11: RMSE = 9.3023, R² = 0.9284
Run 12: RMSE = 10.0210, R² = 0.9173
Run 13: RMSE = 8.8528, R² = 0.9333
Run 14: RMSE = 8.3912, R² = 0.9395
Run 15: RMSE = 8.6283, R² = 0.9383
Run 16: RMSE = 9.4258, R² = 0.9228
Run 17: RMSE = 8.6154, R² = 0.9369
Run 18: RMSE = 8.6523, R² = 0.9362
Run 19: RMSE = 9.4458, R² = 0.9222
Run 20: RMSE = 8.6627, R² = 0.9346
Run 21: RMSE = 9.1779, R² = 0.9285
Run 22: RMSE = 9.7199, R² = 0.9196
Run 23: RMSE = 8.7445, R² = 0.9354
Run 24: RMSE = 9.4600, R² = 0.9249
Run 25: RMSE = 8.5770, R² = 0.9372

Average RMSE over 25 runs: 8.9311
Average R² over 25 runs: 0.9317


Run 1: RMSE = 8.3812, R² = 0.9387
Run 2: RMSE = 9.0486, R² = 0.9285
Run 3: RMSE = 8.8819, R² = 0.9314
Run 4: RMSE = 8.6493, R² = 0.9355
Run 5: RMSE = 9.3036, R² = 0.9249
Run 6: RMSE = 8.8660, R² = 0.9303
Run 7: RMSE = 8.9372, R² = 0.9323
Run 8: RMSE = 8.8508, R² = 0.9324
Run 9: RMSE = 8.3508, R² = 0.9409
Run 10: RMSE = 8.3315, R² = 0.9414
Run 11: RMSE = 9.3023, R² = 0.9284
Run 12: RMSE = 10.0210, R² = 0.9173
Run 13: RMSE = 8.8528, R² = 0.9333
Run 14: RMSE = 8.3912, R² = 0.9395
Run 15: RMSE = 8.6283, R² = 0.9383
Run 16: RMSE = 9.4258, R² = 0.9228
Run 17: RMSE = 8.6154, R² = 0.9369
Run 18: RMSE = 8.6523, R² = 0.9362
Run 19: RMSE = 9.4458, R² = 0.9222
Run 20: RMSE = 8.6627, R² = 0.9346
Run 21: RMSE = 9.1779, R² = 0.9285
Run 22: RMSE = 9.7199, R² = 0.9196
Run 23: RMSE = 8.7445, R² = 0.9354
Run 24: RMSE = 9.4600, R² = 0.9249
Run 25: RMSE = 8.5770, R² = 0.9372

Average RMSE over 25 runs: 8.9311
Average R² over 25 runs: 0.9317

In [4]:
# now train and test the new model ONLY on the original features
# use the original 66/33 train/test split and evaluate performance of 25 runs, each with different random samples


import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load the data
main_data = pd.read_csv("./data/train.csv")

# 'critical_temp' is the target
X = main_data.drop('critical_temp', axis=1)
y = main_data['critical_temp']


# Optimized LightGBM Model
optimized_lgb = lgb.LGBMRegressor(n_estimators=496, max_depth=15, learning_rate=0.057878589503943714, 
                                  subsample=0.6619352139576826, colsample_bytree=0.7512301369524537, 
                                  num_leaves=148, force_col_wise=True, verbose=-1, random_state=42)


# Optimized XGBoost Model
optimized_xgb = xgb.XGBRegressor(n_estimators=407, max_depth=10, learning_rate=0.02962746174406205,
                                 subsample=0.8786056663685927, colsample_bytree=0.6260167856358314,
                                 gamma=4.321388407974591, tree_method="hist", random_state=42)


# Define blending weights
best_weight_lgb = 0.3454  # Previously found optimal weight
best_weight_xgb = 1.0 - best_weight_lgb


# prepare for doing 25 runs as in the original paper
n_runs = 25
rmse_list = []
r2_list = []

for i in range(n_runs):
    # Perform a 66/33 random split; vary the random_state for each iteration
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42 + i)
    
    # Fit the model on the training set
    optimized_lgb.fit(X_train, y_train)
    optimized_xgb.fit(X_train, y_train)

    # Predict on the test set
    y_pred_lgb_test = optimized_lgb.predict(X_test)
    y_pred_xgb_test = optimized_xgb.predict(X_test)
    y_pred = (best_weight_lgb * y_pred_lgb_test) + (best_weight_xgb * y_pred_xgb_test)
    
    # Compute RMSE and R² for this run
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    rmse_list.append(rmse)
    r2_list.append(r2)
    
    print(f"Run {i+1}: RMSE = {rmse:.4f}, R² = {r2:.4f}")

# Compute the average RMSE and R² over the 25 runs
avg_rmse = np.mean(rmse_list)
avg_r2 = np.mean(r2_list)
print(f"\nAverage RMSE over 25 runs: {avg_rmse:.4f}")
print(f"Average R² over 25 runs: {avg_r2:.4f}")




Run 1: RMSE = 9.2869, R² = 0.9258
Run 2: RMSE = 9.3447, R² = 0.9254
Run 3: RMSE = 9.2280, R² = 0.9274
Run 4: RMSE = 9.2769, R² = 0.9262
Run 5: RMSE = 9.3838, R² = 0.9245
Run 6: RMSE = 9.4203, R² = 0.9251
Run 7: RMSE = 9.4368, R² = 0.9238
Run 8: RMSE = 9.2680, R² = 0.9257
Run 9: RMSE = 9.1635, R² = 0.9284
Run 10: RMSE = 9.4050, R² = 0.9252
Run 11: RMSE = 9.3046, R² = 0.9262
Run 12: RMSE = 9.5727, R² = 0.9237
Run 13: RMSE = 9.2660, R² = 0.9259
Run 14: RMSE = 9.0604, R² = 0.9302
Run 15: RMSE = 9.1400, R² = 0.9289
Run 16: RMSE = 9.3216, R² = 0.9262
Run 17: RMSE = 9.1512, R² = 0.9286
Run 18: RMSE = 9.4491, R² = 0.9243
Run 19: RMSE = 9.2136, R² = 0.9279
Run 20: RMSE = 9.1873, R² = 0.9283
Run 21: RMSE = 9.4156, R² = 0.9252
Run 22: RMSE = 9.4808, R² = 0.9232
Run 23: RMSE = 9.3287, R² = 0.9250
Run 24: RMSE = 9.4746, R² = 0.9235
Run 25: RMSE = 9.3616, R² = 0.9250

Average RMSE over 25 runs: 9.3177
Average R² over 25 runs: 0.9260


Results original features with 66/33 split on new model:

Run 1: RMSE = 9.2869, R² = 0.9258
Run 2: RMSE = 9.3447, R² = 0.9254
Run 3: RMSE = 9.2280, R² = 0.9274
Run 4: RMSE = 9.2769, R² = 0.9262
Run 5: RMSE = 9.3838, R² = 0.9245
Run 6: RMSE = 9.4203, R² = 0.9251
Run 7: RMSE = 9.4368, R² = 0.9238
Run 8: RMSE = 9.2680, R² = 0.9257
Run 9: RMSE = 9.1635, R² = 0.9284
Run 10: RMSE = 9.4050, R² = 0.9252
Run 11: RMSE = 9.3046, R² = 0.9262
Run 12: RMSE = 9.5727, R² = 0.9237
Run 13: RMSE = 9.2660, R² = 0.9259
Run 14: RMSE = 9.0604, R² = 0.9302
Run 15: RMSE = 9.1400, R² = 0.9289
Run 16: RMSE = 9.3216, R² = 0.9262
Run 17: RMSE = 9.1512, R² = 0.9286
Run 18: RMSE = 9.4491, R² = 0.9243
Run 19: RMSE = 9.2136, R² = 0.9279
Run 20: RMSE = 9.1873, R² = 0.9283
Run 21: RMSE = 9.4156, R² = 0.9252
Run 22: RMSE = 9.4808, R² = 0.9232
Run 23: RMSE = 9.3287, R² = 0.9250
Run 24: RMSE = 9.4746, R² = 0.9235
Run 25: RMSE = 9.3616, R² = 0.9250

Average RMSE over 25 runs: 9.3177

Average R² over 25 runs: 0.9260